In [1]:
import requests
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import os

In [2]:
def get_ip():
    response = requests.get('https://api64.ipify.org?format=json').json()
    return response["ip"]


def get_location():
    ip_address = get_ip()
    response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()
    location_data = {
        "ip": ip_address,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name")
    }
    return location_data

In [3]:
print(get_location())

{'ip': '208.85.234.131', 'city': 'Springfield', 'region': 'Oregon', 'country': 'United States'}


In [46]:
def get_zip(x):
    response = requests.get(f'https://ipapi.co/{x}/json/').json()
    res = response.get('postal')
    if type(res) is str:
        return int(res)
    else:
        return None

In [4]:
ip_address = get_ip()

In [18]:
ip_address = '73.240.110.67'

In [19]:
response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()

In [44]:
type(response.get('postal')) is str

True

In [22]:
path = r'T:\DCProjects\Support\Lane'

In [77]:
dat = pd.read_csv(os.path.join(path, 'TransitPlanningforRuralLaneCounty.csv'))

In [78]:
dat.loc[:, 'ZipRev'] = dat.copy()['IPAddress'].apply(lambda x: get_zip(x))

In [32]:
def revZipCode(x, y):
    if x == y:
        res = 0
    else:
        res = 1
    return res

In [34]:
dat.columns

Index(['RespondentID', 'CollectorID', 'StartDate', 'EndDate', 'IPAddress',
       'ZipCode', 'Area', 'ZipRev'],
      dtype='object')

In [48]:
dat.loc[:, 'Diff'] = dat.copy()[['ZipCode', 'ZipRev']].apply(lambda row: revZipCode(row.ZipCode, row.ZipRev), axis=1)

In [38]:
dat.shape

(194, 9)

In [76]:
dat.loc[dat.Diff==1, ['IPAddress', 'ZipCode', 'ZipRev']]

AttributeError: 'GeoDataFrame' object has no attribute 'Diff'

In [59]:
df = data[['RespondentID', 'ZipCode']].groupby(['ZipCode']).agg('count').reset_index()

In [64]:
df['ZipCode'] = df['ZipCode'].apply(str)

In [51]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

zip_sql = '''
SELECT 
zipcode AS ZipCode,
mailcity AS City,
Shape.STAsBinary() AS geometry
FROM dbo.ZIPCode;
'''

In [52]:
def readZipCode():
    ZipCode = gpd.GeoDataFrame.from_postgis(zip_sql, engine, geom_col='geometry')
    ZipCode.crs = "EPSG:2914"
    return ZipCode

In [53]:
zip_shp = readZipCode()

In [63]:
zip_shp.ZipCode.values[0]

'97324'

In [69]:
data = zip_shp.merge(df, on='ZipCode', how='left')

In [70]:
data.rename(columns={"RespondentID": "IDcount"}, inplace=True)

In [74]:
data.loc[data.IDcount.isnull(), 'IDcount'] = 0

In [75]:
data.to_file(os.path.join(path, 'Survey_Count.shp'))